In [3]:
!pip install pandasql
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sklearn
import scipy

  Stored in directory: C:\Users\hipra\AppData\Local\pip\Cache\wheels\53\6c\18\b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [5]:
from sklearn.datasets import load_iris
import pandas as pd
from pandasql import sqldf
from pandasql import load_meat, load_births
import re

In [6]:
births = load_births()
meat = load_meat()
iris = load_iris()


iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)

iris_df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)

iris_df.columns = [re.sub("[() ]", "", col) for col in iris_df.columns]

In [7]:
print(sqldf("SELECT * FROM iris_df LIMIT 10;", locals()))

print(sqldf("SELECT sepalwidthcm, species FROM iris_df LIMIT 10;", locals()))

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm species
0            5.1           3.5            1.4           0.2  setosa
1            4.9           3.0            1.4           0.2  setosa
2            4.7           3.2            1.3           0.2  setosa
3            4.6           3.1            1.5           0.2  setosa
4            5.0           3.6            1.4           0.2  setosa
5            5.4           3.9            1.7           0.4  setosa
6            4.6           3.4            1.4           0.3  setosa
7            5.0           3.4            1.5           0.2  setosa
8            4.4           2.9            1.4           0.2  setosa
9            4.9           3.1            1.5           0.1  setosa
   sepalwidthcm species
0           3.5  setosa
1           3.0  setosa
2           3.2  setosa
3           3.1  setosa
4           3.6  setosa
5           3.9  setosa
6           3.4  setosa
7           3.4  setosa
8           2.9  setosa
9           

In [8]:
q = """
      select
        species
        , avg(sepalwidthcm)
        , min(sepalwidthcm)
        , max(sepalwidthcm)
      from
        iris_df
      group by
        species;
        
"""
print("*" * 80)
print("aggregation")
print("-" * 80)
print(q)
print(sqldf(q, locals()))

********************************************************************************
aggregation
--------------------------------------------------------------------------------

      select
        species
        , avg(sepalwidthcm)
        , min(sepalwidthcm)
        , max(sepalwidthcm)
      from
        iris_df
      group by
        species;
        

      species  avg(sepalwidthcm)  min(sepalwidthcm)  max(sepalwidthcm)
0      setosa              3.428                2.3                4.4
1  versicolor              2.770                2.0                3.4
2   virginica              2.974                2.2                3.8


In [9]:
def pysqldf(q):
    "add this to your script if you get tired of calling locals()"
    return sqldf(q, globals())


print("*" * 80)
print("calling from a helper function")
print('''def pysqldf(q):)
    "add this to your script if you get tired of calling locals()"
        return sqldf(q, globals())''')
print("-" * 80)
print(q)
print(pysqldf(q))

********************************************************************************
calling from a helper function
def pysqldf(q):)
    "add this to your script if you get tired of calling locals()"
        return sqldf(q, globals())
--------------------------------------------------------------------------------

      select
        species
        , avg(sepalwidthcm)
        , min(sepalwidthcm)
        , max(sepalwidthcm)
      from
        iris_df
      group by
        species;
        

      species  avg(sepalwidthcm)  min(sepalwidthcm)  max(sepalwidthcm)
0      setosa              3.428                2.3                4.4
1  versicolor              2.770                2.0                3.4
2   virginica              2.974                2.2                3.8


In [10]:
q = """
    select
        a.*
    from
        iris_df a
    inner join
        iris_df b
            on a.species = b.species
    limit 10;
"""

print("*" * 80)
print("joins")
print("-" * 80)
print(q)
print(pysqldf(q))

********************************************************************************
joins
--------------------------------------------------------------------------------

    select
        a.*
    from
        iris_df a
    inner join
        iris_df b
            on a.species = b.species
    limit 10;

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm species
0            5.1           3.5            1.4           0.2  setosa
1            5.1           3.5            1.4           0.2  setosa
2            5.1           3.5            1.4           0.2  setosa
3            5.1           3.5            1.4           0.2  setosa
4            5.1           3.5            1.4           0.2  setosa
5            5.1           3.5            1.4           0.2  setosa
6            5.1           3.5            1.4           0.2  setosa
7            5.1           3.5            1.4           0.2  setosa
8            5.1           3.5            1.4           0.2  setosa
9            5.1

In [18]:
q = """
    select
        *
    from
        iris_df
    where
        species = 'virginica'
        and sepallengthcm > 7.7;
"""
print("*" * 80)
print("where clause")
print("-" * 80)
print(q)
print(pysqldf(q))

********************************************************************************
where clause
--------------------------------------------------------------------------------

    select
        *
    from
        iris_df
    where
        species = 'virginica'
        and sepallengthcm > 7.7;

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm    species   id
0            7.9           3.8            6.4           2.0  virginica  131


In [12]:
iris_df['id'] = range(len(iris_df))
q = """
    select
        *
    from
        iris_df
    where
        id in (select id from iris_df where sepalwidthcm*sepallengthcm > 25);
"""
print("*" * 80)
print("subqueries")
print("-" * 80)
print(q)
print(pysqldf(q))

********************************************************************************
subqueries
--------------------------------------------------------------------------------

    select
        *
    from
        iris_df
    where
        id in (select id from iris_df where sepalwidthcm*sepallengthcm > 25);

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm    species   id
0            5.7           4.4            1.5           0.4     setosa   15
1            7.2           3.6            6.1           2.5  virginica  109
2            7.7           3.8            6.7           2.2  virginica  117
3            7.9           3.8            6.4           2.0  virginica  131


In [13]:
q = """
    SELECT
        m.*
        , b.births
    FROM
        meat m
    INNER JOIN
        births b
            on m.date = b.date
    ORDER BY
        m.date;
"""

print(pysqldf(q).head())

                         date    beef  veal    pork  lamb_and_mutton  \
0  1975-01-01 00:00:00.000000  2106.0  59.0  1114.0             36.0   
1  1975-02-01 00:00:00.000000  1845.0  50.0   954.0             31.0   
2  1975-03-01 00:00:00.000000  1891.0  57.0   976.0             35.0   
3  1975-04-01 00:00:00.000000  1895.0  60.0  1100.0             34.0   
4  1975-05-01 00:00:00.000000  1849.0  59.0   934.0             31.0   

   broilers  other_chicken  turkey  births  
0     646.2            NaN    64.9  265775  
1     570.2            NaN    47.1  241045  
2     616.6            NaN    54.4  268849  
3     688.3            NaN    68.7  247455  
4     690.1            NaN    81.9  254545  


In [14]:
births

,date,births
0,1975-01-01,265775
1,1975-02-01,241045
2,1975-03-01,268849
3,1975-04-01,247455
4,1975-05-01,254545
5,1975-06-01,254096
6,1975-07-01,275163
7,1975-08-01,281300
8,1975-09-01,270738
9,1975-10-01,265494


In [15]:
meat

,date,beef,veal,pork,lamb_and_mutton,broilers,other_chicken,turkey
0,1944-01-01,751.0,85.0,1280.0,89.0,NaN,NaN,NaN
1,1944-02-01,713.0,77.0,1169.0,72.0,NaN,NaN,NaN
2,1944-03-01,741.0,90.0,1128.0,75.0,NaN,NaN,NaN
3,1944-04-01,650.0,89.0,978.0,66.0,NaN,NaN,NaN
4,1944-05-01,681.0,106.0,1029.0,78.0,NaN,NaN,NaN
5,1944-06-01,658.0,125.0,962.0,79.0,NaN,NaN,NaN
6,1944-07-01,662.0,142.0,796.0,82.0,NaN,NaN,NaN
7,1944-08-01,787.0,175.0,748.0,87.0,NaN,NaN,NaN
8,1944-09-01,774.0,182.0,678.0,91.0,NaN,NaN,NaN
9,1944-10-01,834.0,215.0,777.0,100.0,NaN,NaN,NaN


In [16]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  